<a href="https://colab.research.google.com/github/sergiokapone/DataScience/blob/main/Hw7/Hw7_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання
Візьміть датасет [movielens](https://surprise.readthedocs.io/en/stable/dataset.html) і побудуйте модель матричної факторизації. У даній бібліотеці він має назву SVD. Підберіть найкращі параметри за допомогою крос-валідації, також поекспериментуйте з іншими [алгоритмами](https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html) розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним.


In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163333 sha256=9335fb54bb2bb61dee2964ef384be9a76a686a0c74832801819c34f2c05643d2
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
from surprise import SVD, SVDpp, NMF
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split, cross_validate
from hyperopt import hp, fmin, tpe, Trials


In [ ]:
# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

In [ ]:
# sample random trainset and testset
# test set is made of 25% of the ratings.
train_set, test_set = train_test_split(data, test_size=0.25)

In [ ]:
space_svd = {
    'n_factors': hp.choice('n_factors', [50, 100, 150]),
    'n_epochs': hp.choice('n_epochs', [10, 20, 30]),
    'lr_all': hp.uniform('lr_all', 0.001, 0.1),
    'reg_all': hp.uniform('reg_all', 0.01, 0.2),
    'random_state': 42
}

space_nmf = {
        'n_factors': hp.choice('n_factors', [5, 10, 15, 20, 25]),
        'n_epochs': hp.choice('n_epochs', [10, 20, 30, 40, 50]),
        'biased': hp.choice('biased', [True, False]),
        'reg_pu': hp.uniform('reg_pu', 0.001, 0.1),
        'reg_qi': hp.uniform('reg_qi', 0.001, 0.1),
        'lr_bu': hp.uniform('lr_bu', 0.001, 0.1),
        'lr_bi': hp.uniform('lr_bi', 0.001, 0.1),
        'random_state': 42
    }

In [ ]:
# Функція для обчислення RMSE
def objective(params, func):
    model = func(**params)
    model.fit(train_set)
    predictions = model.test(test_set)
    rmse = accuracy.rmse(predictions)
    return rmse

best_params = []
for i, func in enumerate([SVD, SVDpp, NMF]):
    print('-------------------------')
    print(f'Srart: {func.__name__}')
    print('-------------------------')
    space=[space_svd, space_svd, space_nmf]

    best = fmin(fn=lambda params: objective(params, func), space=space[i], algo=tpe.suggest, max_evals=2)
    best_params.append({func.__name__: best})

best_params

In [ ]:
# Побудова моделі SVD
model_svdpp = SVD(**best_params[0].get('SVD'))
results_svdpp = cross_validate(model_svdpp, data, measures=['MSE', 'MAE'], cv=5, verbose=True)

In [ ]:
# Побудова моделі NMF
model_nmf = NMF(**best_params[2].get('NMF'))
results_nmf = cross_validate(model_nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
# Побудова моделі SVDpp
model_svdpp = SVDpp(**best_params[1].get('SVDpp'))
results_nmf = cross_validate(model_svdpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)